In [1]:
include("../Algebra/Gradients.jl")
include("../Algebra/Hamiltonians.jl")
include("../Algebra/Matrices.jl")
include("../Algebra/Propagators.jl")
include("../Amplitudes/Chebyshev.jl")
include("../Costs/Costs.jl")
include("../Gates/Xgate.jl")

U2 (generic function with 1 method)

In [4]:
using LinearAlgebra, Optim
using BenchmarkTools, Distributions

In [24]:
ω_c = 5
ω_q = 2
χ = 3e-3
N = 6
amp_c = 1
amp_q = 1

1

In [25]:
I_qubit = Matrix(I,2,2)
I_cavity = Matrix(I,N,N)
X_gate = kron(x_gate(N, Array[[2,5]]), I_qubit)

#X_gate = kron(x_gate(N, Array[[2,3], [4,5]]), I_qubit)

θ = π/2.5
RZ = kron(Rz(N, Array[[3,4]], θ/2), I_qubit)
RX = kron(Rx(N, Array[[3,4]], θ/4), I_qubit)
Udue = kron(U2(N, Array[[3,4]], [θ/2 θ/3 θ/4] ), I_qubit)

sx = kron(I_cavity, [0 1; 1 0])
sy = kron(I_cavity, [0 -1im; 1im 0])

ψ_initial = rand(N)
ψ_initial = kron(ψ_initial/norm(ψ_initial), [0,1])

12-element Vector{Float64}:
 0.0
 0.1410786669167171
 0.0
 0.8417114118685988
 0.0
 0.24527010532759094
 0.0
 0.449277390489757
 0.0
 0.020625646069524665
 0.0
 0.09584202326064188

In [26]:
ψ_final_u = Udue * ψ_initial
ψ_final_x = RX * ψ_initial
ψ_final_x_gate = X_gate * ψ_initial

12-element Vector{Float64}:
 0.0
 0.1410786669167171
 0.0
 0.020625646069524665
 0.0
 0.24527010532759094
 0.0
 0.449277390489757
 0.0
 0.8417114118685988
 0.0
 0.09584202326064188

In [27]:
# generating matrices
a,adag,sp,sm,sz = generate_matrices(N)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0], [0 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 1 0], [1 0 … 0 0; 0 -1 … 0 0; … ; 0 0 … 1 0; 0 0 … 0 -1])

In [28]:
disp(α) = exp((α * adag - α' * a))
ECD(β) = disp(β / 2) * sp + disp(- β / 2) * sm
ROT(ϕ, θ) = exp(-1im * θ / 2 * ( sx * cos(ϕ) + sy * sin(ϕ) )) 
snap(x) = kron(Diagonal(exp.(-1im.*x)),I_qubit)

snap (generic function with 1 method)

In [29]:
function cost(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function cost_ecd_rot_ecd(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ECD(β_i[i] + β_r[i] * 1im) * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)'
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function gradient(coefficients, unitary, N, h, cost_before_increment)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost(coeffs_copy, unitary, N) - cost_before_increment) / h
    end

    return gradient

end

function gate_composition(coefficients, N, l_coeff)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return propagator
    
end

function cost_state_prep(coefficients, N, ψ_n, ψ_m)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(ψ_m' * propagator * ψ_n)^2
    
end


function gradient_state_prep(coefficients, N, h, cost_before_increment, ψ_n, ψ_m)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost_state_prep(coeffs_copy, N, ψ_n, ψ_m) - cost_before_increment) / h
    end

    return gradient

end

gradient_state_prep (generic function with 1 method)

In [30]:
snapgate = snap(rand(N))

12×12 Matrix{ComplexF64}:
 0.815621-0.578587im       0.0-0.0im       …     0.0+0.0im
      0.0-0.0im       0.815621-0.578587im        0.0+0.0im
      0.0+0.0im            0.0+0.0im             0.0+0.0im
      0.0+0.0im            0.0+0.0im             0.0+0.0im
      0.0+0.0im            0.0+0.0im             0.0+0.0im
      0.0+0.0im            0.0+0.0im       …     0.0+0.0im
      0.0+0.0im            0.0+0.0im             0.0+0.0im
      0.0+0.0im            0.0+0.0im             0.0+0.0im
      0.0+0.0im            0.0+0.0im             0.0+0.0im
      0.0+0.0im            0.0+0.0im             0.0+0.0im
      0.0+0.0im            0.0+0.0im       …     0.0-0.0im
      0.0+0.0im            0.0+0.0im          0.7624-0.647106im

In [31]:
f(x) = cost(x, RX, N * 2)
g(x) = cost_state_prep(x, N * 2, ψ_initial, ψ_final_x_gate)

g (generic function with 1 method)

In [32]:
costs = []
for depth in 30:6:36
    it_costs = []
    for iteration in 1:1
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        # coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/2)]..., (rand(Int(l_coeff / 2 - length(opt_coefficients)/2))*2 .-1)...]
        # coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:end]..., zeros(Int(l_coeff / 2 - length(opt_coefficients)/2))...]
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        res = Optim.optimize(f, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=200, iterations = 1000))
        push!(it_costs,Optim.minimum(res))
        println(Optim.minimum(res))
        println(Optim.minimizer(res))
    end
    println(minimum(it_costs))
    push!(costs,minimum(it_costs))
end

blocks: 30 iterations :1
Iter     Function value   Gradient norm 


In [13]:
RX = kron(Rx(N, Array[[3,4]], θ*2.5), I_qubit)

12×12 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  1.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  1.0+0.0im

In [18]:
f(x) = cost(x, RX, N * 2)
costs = []
for depth in 36:6:36
    it_costs = []
    for iteration in 1:3
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        # coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/2)]..., (rand(Int(l_coeff / 2 - length(opt_coefficients)/2))*2 .-1)...]
        # coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:end]..., zeros(Int(l_coeff / 2 - length(opt_coefficients)/2))...]
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        res = Optim.optimize(f, coefficients , LBFGS(), Optim.Options())
        push!(it_costs,Optim.minimum(res))
        println(Optim.minimum(res))
        println(Optim.minimizer(res))
    end
    println(minimum(it_costs))
    push!(costs,minimum(it_costs))
end

blocks: 36 iterations :1
0.00025857461534972703


[1.0466535160216484, 0.039069188934231086, -0.7247506337569685, -0.06667487265380571, -1.1724881630811657, -0.7307163592776275, -0.9121555146407051, -0.6620076932100785, -0.45815230337565255, 0.03697651862340152, -0.49059364438005987, 0.07741503430280619, 0.056730162524362984, 1.3855688455235153, -1.5851293823158197, -0.4019668323530738, -0.7431329482277762, -0.9207447020109353, -0.01749263137343902, 1.0264694735718058, 0.460174226612208, -0.8102032838207024, -0.6694798104129163, 0.37614424482994996, 0.2797037903809828, 1.214053972257878, 0.3584085781041275, -0.23570245270179013, 0.7177932836190233, 0.156455596219713, -0.7338298518699984, -0.6832345793801755, -0.004351713778191889, -1.0947404194272563, -0.40556669154930275, 0.5364329373241018, -0.1897708990486698, 1.0959876494775582, 0.5347363998878207, -0.8689021195037105, 0.8795963797819606, 0.47023986679941887, -0.6519802660720501, 0.8835297690707996, -0.9352448727460061, 1.7196122532346718, -0.9709343045809171, -0.4281203062630826

3.1846486915676486e-5
[0.03740024862369406, -0.3465158758955572, -1.7613352527860409, 0.8703932718322251, -0.14601224813540445, -0.5820756756170353, 0.2083649769560322, 0.8720686387934184, 0.4602844783820326, -1.4252834267318704, 1.0087928582173231, -1.34076395812819, -0.9561526588535273, 1.477081617493388, -0.5859227871045636, -1.7924050808690248, 0.649412132694054, 0.5034135150059003, -0.9704330900243656, 1.136235720851052, -0.253447552589307, 1.0023712778080685, 0.9720914124525313, -0.8336321018933782, 1.8351018015764153, -0.4827231709632174, 0.8931516841317375, 0.6659295016676225, 0.5834615680614289, 0.5470528741694847, -0.5064276294788084, -0.5282173988409066, -0.04893192169277993, 0.6404473579097549, 0.2952634463498198, 0.24103387630674988, 1.0101540385388437, -0.5856218164723431, -0.009749504470660869, -2.3487792973170576, -0.5716784701174521, 1.2252558077987508, -0.7483144396231938, 0.4734367847255555, -0.6452812299859257, -0.3640758267707079, -0.22552672467706258, -1.152941023

0.0005621550460743752
[0.5354283890865336, 1.0599467900135149, 0.5155499009411637, -1.288207157241871, 0.6767806879446662, -0.44844022333412026, -0.36122397967620984, -0.5665164571253168, 0.07947041105877019, -0.060756184044738946, -0.14184308745644134, -0.02553917063567122, -1.2922305828885587, -0.2992262252500485, -0.38186190463648373, -0.5203207229877626, -1.2192706154724442, -0.2459149564236067, -1.0327646446252197, 1.0423174035712277, 0.1125402890306205, 1.9531896458144258, 0.6301888160502854, 0.020997826935666295, 0.4551465926532869, -1.124178738667393, 0.15830299074369986, 1.2925657383456697, -0.06394879293337573, -0.2335191116857816, 0.08561584749189635, 0.20953800092205577, -0.9483401279808659, -0.09829874937721816, -0.8002112148456922, 0.5247617426211234, 0.6829007309924519, 0.6264017445711988, -0.6413211951746914, 0.33319634658673497, 0.1691153316078175, 0.5583689759667175, 0.9391690281771725, 1.6581771457943821, 1.505403671096557, 0.6122748769381726, 0.8953379506921485, 1.3

In [20]:
println("X 2,5")

f(x) = cost(x, X_gate, N * 2)
costs = []
for depth in 18:6:36
    it_costs = []
    for iteration in 1:1
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        # coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/2)]..., (rand(Int(l_coeff / 2 - length(opt_coefficients)/2))*2 .-1)...]
        # coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:end]..., zeros(Int(l_coeff / 2 - length(opt_coefficients)/2))...]
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        res = Optim.optimize(f, coefficients , LBFGS(), Optim.Options())
        push!(it_costs,Optim.minimum(res))
        println(Optim.minimum(res))
        println(Optim.minimizer(res))
    end
    println(minimum(it_costs))
    push!(costs,minimum(it_costs))
end

X 2,5


blocks: 18 iterations :1
0.1919395292231787


[-0.36060388300115725, -0.8685566901179844, 0.3023834470528745, 0.6149317978143682, -0.922276095568056, -0.4829984046734524, -0.6736721990775313, 0.6351805420348864, -0.5851546627027745, 0.3644448776708854, 1.5067302079682763, -0.8203010693559443, -0.29934209948498797, -0.6094111626392489, 1.9538625317419864, -0.3932534021944943, 1.2600663350188377, 0.40265626839594704, 0.9021200108622376, -0.7606345156340949, 0.4224252492640091, -0.348141403509353, 1.5443934046800105, 1.1961296066011669, 0.18028149597224502, -1.0147812262267435, -0.8400140493626588, 1.0196509642179639, -0.10983806070983274, -0.9261697871068473, 1.0195136296070697, -1.1436083526302308, -1.1919352871526863, -0.5278229851291695, 0.25715800047511683, -0.7489282193948719, -1.223613107329794, -3.778098165172551, 1.7546591902442241, -1.3027361018557266, -4.0839087170207184, 2.125149639456385, 2.5486476543172216, 1.4466278092022749, -1.1882467751658006, 1.95003155361707, -2.010785844174352, -1.8172516737148878, 1.91914561883

0.04058165890136378
[-0.22429474745338698, -1.4156648433898171, -1.259148256618502, 0.749686504879005, 1.2869063626234063, 0.7983999683312897, -0.9458848954173196, -0.4530448760791564, -1.1283513044557327, 0.2570268816920115, 2.3343573924360648, 0.033169268313187346, -1.9528164207277858, 0.4762084352285962, -0.8341475353924012, -1.8113416671629148, -1.6084720757189346, 0.43440732431610257, 0.15425402311943592, 0.4791746367686874, 1.4528048934532458, -3.027611797575874, -0.659717149265513, -1.1292791608374972, 0.2660673540593648, -0.7649638530880818, 0.10861025398101182, 0.28045738366805745, 1.7926901265241821, -0.5324334214513364, -0.15879419645157616, 1.5766038446336557, 0.05429069217163192, 1.0685823608282847, -0.5063152188386593, -1.2655562759537968, -0.9464019520596608, 1.4243201230675102, 0.5983821885790261, -1.1191797076269543, 0.46593663459196816, 0.629754165279599, -1.2520782666881811, -0.1539996863877829, 0.30297859152033135, -0.17915046489909614, 0.5312425665591746, -0.922407

0.0067171012189508605
[0.7367347324717077, -0.9615320635984729, 0.08329416696891899, 0.09878610472220323, 0.6815660817509568, 1.0197198924129862, -0.7106979508358167, -0.5939320705885806, 0.840955952244534, 0.04418073401290922, 0.9551533912210659, -2.3661734889521218, 0.9821028856805987, 1.2214347781017791, -0.7895792864624386, -0.45673918421413445, -0.628821619375615, 0.4523828627594398, 1.6826138652769917, -0.11349832103784399, -0.9785834530660645, -0.7964471258521588, -1.5767465533272904, -1.0772451260498912, -1.4380291312669033, 0.8737077916514449, 0.2793004154885841, 1.7192879232348337, 0.020198020409941512, -1.5722842360225067, -0.3985179078808251, 1.1524717889485154, -0.9743493499272937, -0.4401053198074331, -0.5564086852624556, -0.23541891984584162, 0.590874276057971, 0.993666223496117, 1.1581586714530168, 0.8359038636532724, -0.00045583692483836983, 0.6224001231193609, -1.3315560387024092, -0.3746991988339849, -1.2189118831582255, 0.7998991752320976, 1.9063730481600394, 1.3088

1.1289509489476046e-5
[0.278269338150751, -0.846981663715944, 0.9422645354132471, -1.3686979484314583, -0.9658442643668274, 0.46392348975747283, -0.08662926664417725, 1.5680076417598405, 1.8491950907934842, -1.8643414414062052, 0.5575135496468759, 0.8054343872375489, -1.4984381912634686, -0.6094442298689478, -1.0536091802499234, -0.4205279799129043, -0.48500614823201804, 1.055196935446125, 0.5056875615039665, 0.5002702550343635, -1.0489465877887645, 1.3412151941142203, -0.655266194383379, -1.1756319851647081, -0.3322127803420149, 0.37242180352733895, -0.08599401727656215, 0.7982529354184678, -1.147504040622886, -0.2211604230113636, 0.7645346666740369, 0.27692416559301497, 1.436388011439935, 2.023402719635742, -0.6590182469184768, -0.5722304781459231, 0.6173738347283239, 0.7326161831110154, 0.3914613336698129, -0.8547808065578554, -1.3376869073608586, 1.217182102845406, 1.4125936555718184, -2.0330853253762173, 0.19636693812587022, 0.46135740779757933, -1.2093263976539104, 1.408376719384

# stoc gradient descent

In [149]:
costs_final = []
for depth in 64:8:64
    it_costs = []
    for iteration in 1:3
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        coefficients = [0.8690408632149678, -0.7236416401971805, -0.0908719850588177, 0.8258628776740167, 0.9056445769487957, -0.8220939210081029, 0.29601861853616424, 0.7813552229187792, 0.9837594907380149, -0.10686709645389225, -0.6656407250459772, 0.992278727387623, -0.4893461915705191, -1.093993962903741, 0.4845407107960842, -0.33857113975897773, -0.438797994554033, -0.5977365084149797, 0.20827466739249778, -0.8959109454265681, 0.5383716212207733, 1.3219169845803183, 0.8578064725920909, -0.7256810259845291, 0.37256259960242327, -1.2114316020995235, -0.282902962907945, -0.26590734759323503, 0.5548879566071089, 0.6803304120122661, -1.0549066677248888, 0.4013045273205048, 0.039560949181603355, 1.0529793828938236, -0.044290684794351255, -0.21790564460826214, 0.7641852601374161, 0.9310403877890154, -0.24519813978115026, 0.6528372052655407, -1.2521949229101696, -0.21682088705693697, 0.10911919627652335, -0.31451889968452584, 0.07428438971871276, 0.8168118972903352, -0.4306959382326172, -1.31201487771103, -0.5551956081923786, -0.24938592347753555, 0.7261072879386111, 0.3449918944429117, 1.0218022420859423, 0.9586632779953215, 0.9830015979456759, 1.1800344827793208, -0.6015729201693757, 0.703237921035128, -0.8659071933834304, 0.7405471254766851, 0.5502125342538493, -0.1279261558044507, 0.5436405284887846, 0.7561715320031175, -1.8294742583884145, 0.8274849373492085, -0.5023928927013781, -0.2106775708083146, -0.25623844263590434, -0.13700323024471564, -0.5427032401830472, 0.8481733035397572, 2.857123893083541, 0.6722772567617452, 3.3393904892971857, -2.7921664352181588, -1.4927954172559996, 0.3500341652427946, 2.178592973947075, 2.0048971146113073, -1.3350642362244973, 0.9711032282505817, 2.3816703973089663, -2.6138104747765247, 2.5134841942159674, -1.9047337097507626, 2.2260526473073146, -1.8796539188004344, 1.6517968966982153, 1.4620145034237324, 1.8366376463164558, -1.3614199467978896, -1.3067881525922014, -0.8271088962045482, -1.3523594955345797, -0.8216607978872826, -1.015386302411769, -1.4951025575413324, 2.199130944918348, -2.098541424724332, 1.8106459325712736, -2.7528680557326783, 1.434522172097638, 0.23235089810469334, -1.3427653914596476, -2.1865004319117882, -4.499311913607152, 3.5233665940145906, 0.11147763894762594, 1.7253123134995119, 0.3069539608771961, 0.7456729443236856, 0.9675539799184902, -2.0807423878078692, -0.21274877285924562, -2.4625967782327733, -0.8958201449217154, -0.16506248177386307, -0.7781344488152088, 0.5657089824636788, 0.49103015856219095, -1.893089322888448, 1.8586226474868552, 1.3367905090651682, -0.845322993521433, -2.414547922619251, -2.0653040809018646, 2.13700608386267, 0.6560460356595541, 1.2398223292845443, 2.962599533837159, 1.1325322497723143, 3.463842619105753, -3.938053691752728, -0.9569023982656585, 1.605238908260515, 0.6974820952130029, 2.5406564651431, 2.285077612710168, 0.027961409188739638, 1.0736422395589698, 2.4870006619787572, 0.692378916991639, -2.718696870022777]
        beta1 = 0.8
        beta2 = 0.999
        alpha = 0.01
        m = zeros(length(coefficients))
        v = zeros(length(coefficients))
        coeffs_stored = []
        costs = []
        cost_prev_it = cost(coefficients, RX, N*2)  #cost_state_prep(coefficients, N * 2, ψ_initial, ψ_final_x_gate) #

        for i in 1:1000
            grad = gradient(coefficients, RX, N * 2, 1e-8, cost_prev_it) #gradient_state_prep(coefficients, N*2, 1e-8, cost_prev_it, ψ_initial, ψ_final_x_gate) #
            for n in 1:length(coefficients)
                m[n] = beta1 * m[n] + ( 1 - beta1 ) * grad[n]
                v[n] = beta2 * v[n] + ( 1 - beta2 ) * grad[n]^2
                m_norm = m[n] / ( 1 - beta1^i )
                v_norm = v[n] / ( 1 - beta2^i )
                coefficients[n] = coefficients[n] - alpha * m_norm / (sqrt(v_norm) + 1e-8)
            end
            cost_prev_it = cost(coefficients, RX, N*2)# cost_state_prep(coefficients, N * 2, ψ_initial, ψ_final_x_gate) #
            push!(costs, cost_prev_it)

            if i % 50 == 0
                if norm(grad) < 1e-6
                    break
                end
                println(i)
                println(cost_prev_it)
            end
        end
        println("finished: d = ", depth)
        println(costs[end])
        push!(it_costs,costs[end])
        println(coefficients)
    end
    println(it_costs)
    println(minimum(it_costs))
    push!(costs_final, minimum(it_costs))
end
costs_final

InterruptException: InterruptException:

In [150]:
coeffs = [0.8328194938452032, -0.8105522750655366, -0.03947596969773574, 0.8150417127415052, 0.9355586497841004, -0.819672727969885, 0.15908257377596594, 0.8969730569787527, 1.0867987077197083, -0.12382796442095499, -0.6769159418813943, 1.0246426230169303, -0.5084405635279222, -1.0268972961310772, 0.42484396266977487, -0.30984213118061454, -0.44195548463552226, -0.6327731382009563, 0.2470520241498077, -0.9046312383937988, 0.5670153531055953, 1.3215582797100167, 0.8729333306332443, -0.78886152481934, 0.3445569548035575, -1.2206696780026396, -0.25471100801485647, -0.3585072716256307, 0.6870581315744397, 0.6545040107617951, -1.0221685943730465, 0.3343131234446875, -0.08486885305074147, 1.16534326489163, -0.08178072747870009, -0.17367128231750495, 0.7301925883287174, 0.7960381150713036, -0.2445176384048139, 0.7450024213033812, -1.2896520166163863, -0.2511261526654848, 0.18899633152030276, -0.3134353271761458, 0.040134442582384806, 0.7718476862874823, -0.4457105371804348, -1.2658170397923836, -0.578778138744985, -0.27405676265384593, 0.6995176637177795, 0.35946077517978764, 1.0020099923271824, 0.9306624492297949, 0.9741238940562686, 1.1608927613144042, -0.5310290099006071, 0.8115833948525197, -0.7784531065547783, 0.7352221959115539, 0.5417078187690723, -0.21113457049160275, 0.5103730707595793, 0.7551026732286986, -1.7846390041521236, 0.8631165439769484, -0.5498701334703305, -0.26861474853829703, -0.27455529220647396, -0.07688130726216859, -0.5631568455670428, 0.8533900226411945, 2.8067345186309858, 0.6785936932829509, 3.3943749838043784, -2.805214370969844, -1.4757375492498623, 0.29692768551759546, 2.281415636662691, 2.160137328769026, -1.307321749251418, 1.033561682283, 2.3601394944015057, -2.696457234103049, 2.4969210954848147, -1.906381025598878, 2.2434123597180444, -1.9770399537034882, 1.690842335222719, 1.535746792078091, 1.8604714721583955, -1.3743895036802214, -1.291663669776218, -0.762668513556034, -1.3646714887473783, -0.7879082846775004, -0.9459496839509586, -1.5081193660443275, 2.163188081396188, -2.0477877934412, 1.8200709265036517, -2.7617245086967235, 1.4249304092894297, 0.24880088065636674, -1.2535266524595052, -2.0585026020721804, -4.576841695056939, 3.5387398533822982, -0.07531806514326735, 1.8453317273419196, 0.42767796713041867, 0.7150419102279323, 0.7474552662694737, -1.6860907528015776, -0.2962280834604321, -2.386788332449464, -1.1771757193252919, 0.09208258390797398, -0.8624876874455844, 0.5131833036461091, 0.6136063613800034, -2.0172431784872105, 1.9575955203123583, 1.3468300140519078, -0.7787319428945259, -2.5374797856832374, -1.9401420236852418, 2.0676338346841168, 0.7064923766586017, 1.1338034698609585, 2.9969098154086633, 1.0459448874008392, 3.4682585115304194, -3.9796395422002675, -0.8624504151631956, 1.4625786737059918, 0.8235301000679373, 2.5250726036878977, 2.085654990962269, -0.17147063484675754, 0.7961241933337927, 2.6289885365160464, 0.6582125941103614, -2.531971259434754]
f(x) = cost(x, X_gate, N*2)
println(f(coeffs))

opt_perturbed = ComplexF64[]

for coeff in coeffs
    perturbed = coeff + (rand()*0.002-0.001)
    push!(opt_perturbed,perturbed)
end

f(opt_perturbed)

0.8889226442425289


0.8889281184515271